In [1]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
import torch

# MSE function 
def MSE(pred, true):
    return np.mean((pred-true)**2)


In [2]:
from InformerAPI import InformerTS
from CrossformerAPI import CrossformerTS
from AutoformerAPI import AutoformerTS
from FedformerAPI import FedformerTS

### Looping all models in one go

In [13]:
datasets = {
    'SYNTHh1': './SYNTHDataset/',
    'DEWINDh_small': './WINDataset/'
}
pred_lens = [24, 48, 168, 336, 720]

models = [InformerTS, CrossformerTS, AutoformerTS, FedformerTS]

In [16]:
# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for dataset_name, dataset_path in datasets.items():
        for pred_len in pred_lens:
            print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

            # High amount of epochs to accomodate all models
            # Early stopping should kick in anyways
            model.fit(
                data=dataset_name,
                data_root_path=dataset_path,
                batch_size=32,
                epochs=50, # very high
                pred_len=pred_len
            )
            predictions = model.predict()

            print(f'\nMoving to next...\n')


Model in use: InformerTS
Training InformerTS on SYNTHh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'informerstack', 'features': 'S', 'target': 'TARGET', 'freq': 'h', 'checkpoints': './informer_checkpoints', 'seq_len': 96, 'label_len': 48, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'factor': 5, 'd_model': 512, 'n_heads': 8, 's_layers': [3, 2, 1], 'd_layers': 2, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'des': 'exp', 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'learning_rate': 0.0001, 'loss': 'mse', 'patience': 3, 'data': 'SYNTHh1', 'root_path': './SYNTHDataset/', 'data_path': 'SYNTHh1.csv', 'train_epochs': 50, 'batch_size': 32, 'pred_len': 24, 'detail_freq': 'h'}
Use GPU: cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 44.35 GiB of which 1024.00 KiB is free. Process 16628 has 42.97 GiB memory in use. Process 31166 has 1.36 GiB memory in use. Of the allocated memory 1.00 GiB is allocated by PyTorch, and 17.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF